# 1. 背景：为什么用交叉熵损失`nn.CrossEntropyLoss`？

分类任务的目标是让模型预测的类别分布接近真实标签。

- 真实标签：通常是一个类别编号（比如 0, 1, 2,...）。

- 预测结果：模型输出的往往是未归一化的分数（logits）。

交叉熵损失（`Cross Entropy Loss`）能度量 预测分布与真实分布之间的差异，预测越准确，损失越小。

# 2. 定义

In [ ]:
torch.nn.CrossEntropyLoss(
    weight=None,
    size_average=None,
    ignore_index=-100,
    reduce=None,
    reduction='mean',
    label_smoothing=0.0
)


| 参数                   | 作用                                                                               |
| -------------------- | -------------------------------------------------------------------------------- |
| **weight**           | 各类别的损失权重（Tensor，形状 = `[num_classes]`）。用于类别不均衡。                                   |
| **ignore\_index**    | 忽略某个类别的损失计算（比如在 NLP 任务里，填充 `PAD` 的标签）。                                           |
| **reduction**        | 指定如何聚合 batch 内损失：<br> - `'mean'`：取平均（默认）<br> - `'sum'`：求和<br> - `'none'`：返回逐样本损失 |
| **label\_smoothing** | 标签平滑，减少过拟合。比如 `0.1` 表示把 10% 的概率分散给其它类别。                                          |


## 使用示例
### (1) 单一样本

In [ ]:
import torch
import torch.nn as nn

loss_fn = nn.CrossEntropyLoss()

# 假设类别数=3，模型输出 logits（未经过 softmax）
logits = torch.tensor([[1.5, 0.5, -1.0]])  # shape = [1, 3]
target = torch.tensor([0])  # 真实类别是第0类

loss = loss_fn(logits, target)
print(loss.item())


- 注意：输入 `logits`，不要提前做` softmax`(模型中不需要定义softmax层，直接输出`logits`（逻辑值）)，因为 `nn.CrossEntropyLoss` 内部已经包含了 `LogSoftmax` + `NLLLoss`。

### (2) 批量输入

In [ ]:
# batch=4, 类别数=3
logits = torch.tensor([
    [2.0, 1.0, 0.1],   # 预测类别 0
    [0.5, 2.5, 0.3],   # 预测类别 1
    [0.1, 0.2, 3.0],   # 预测类别 2
    [1.2, 0.7, 1.0]    # 预测类别 0
])  # shape = [4, 3]

targets = torch.tensor([0, 1, 2, 0])  # 真实标签

loss = loss_fn(logits, targets)
print(loss.item())


## 常见误区

1、不要对 `logits` 先做 `softmax`，否则会导致梯度消失或结果错误。

-   模型输出 → 直接传入 `CrossEntropyLoss`。

2、`target` 必须是类别索引（整型 `Tensor`），不是` one-hot`。

-   比如` target=[2]`，而不是 [0,0,1]。

3、在 `二分类` 时，也推荐用 `CrossEntropyLoss`（而不是 BCE），因为它更数值稳定。